In [1]:
%reload_ext autoreload
%autoreload 2

In [57]:
import boto3
import pandas as pd
from io import StringIO
from toolz import first
import json

In [156]:
def query_mpid(mpid):

    s3 = boto3.client('s3')
    bucket_name = 'nbc-events-partitioned'
    file_name = "viewer_partitions/partition=p0000/p0000_2019W29.parquet"
    sql_stmt = f"""select * from s3object s where mpid = {mpid} """
    print(sql_stmt)

    # making the request
    req = s3.select_object_content(
         Bucket=bucket_name,
         Key=file_name,
         ExpressionType='SQL',
         Expression=sql_stmt,
         InputSerialization={'Parquet': {}},
         OutputSerialization = {'JSON': {}},
        )
    return req
    payload = first(req['Payload'])

    reader = payload['Records']['Payload'].decode().split('\n')
    reader = (json.loads(x) for x in reader if x != '')
    return pd.DataFrame(reader)


In [159]:
ret = query_mpid(mpid=3166073724721267440)


select * from s3object s where mpid = 3166073724721267440 


In [160]:
list(ret['Payload'])

[{'Cont': {}},
 {'Stats': {'Details': {'BytesScanned': 9236700,
    'BytesProcessed': 74241173,
    'BytesReturned': 0}}},
 {'End': {}}]

,batch_id,file,file_idx,asof_dt,mpid,nbc_profile,mvpd,event_name,event_type,platform,...,video_end_type,resume,event_id,session_id,video_duration_watched,event_start_unixtime_ms,session_start_unixtime_ms,resume_time,viewer_partition,__index_level_0__
0,ve_20190719_0007,NBC_20190719195925077258_29471.txt,7115,2019-10-12T16:08:32.255121Z,-9219984973637122020,Unauthenticated,Verizon,Video End,custom_event,tvOS,...,Close,False,-737292595624360192,503092962654025563,0,1563566304040,1563566278169,None,0,-9219984973637122020
1,ve_20190715_0002,NBC_20190715043157253587_64856.txt,1626,2019-10-12T15:02:23.069362Z,-9217234899909710820,Unauthenticated,ATT,Video End,custom_event,roku,...,Close,False,-7029079529407193972,-4497709074946949428,2600,1563164848683,1563159394626,None,0,-9217234899909710820
2,ve_20190716_0000,NBC_20190716005546424598_99135.txt,329,2019-10-12T15:13:56.027305Z,-9217041998308945380,Unauthenticated,Unauthenticated,Video End,custom_event,iOS,...,Close,False,6872682937907796272,-5576338246921898354,37,1562988047907,1562985316671,None,0,-9217041998308945380
3,ve_20190716_0001,NBC_20190716020317148975_00693.txt,738,2019-10-12T15:15:02.749489Z,-9217041998308945380,Unauthenticated,Unauthenticated,Video End,custom_event,iOS,...,Close,False,-6607132132478987365,2746371794431879802,37,1563242300390,1563239066049,None,0,-9217041998308945380
4,ve_20190717_0001,NBC_20190717023127549893_68410.txt,925,2019-10-12T15:29:15.232151Z,-9217041998308945380,Unauthenticated,Unauthenticated,Video End,custom_event,iOS,...,Close,False,7445835722267705731,-409492438800141359,37,1563330590325,1563327716344,None,0,-9217041998308945380


In [85]:
for x in reader:
    if x =='': break